In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
import time
from config import gkey

In [ ]:
neighborhood_df = pd.read_csv("locations.csv", encoding="ISO-8859-1")

In [ ]:
neighborhood_df['Neighborhood Lat'] = ""
neighborhood_df['Neighborhood Long'] = ""

neighborhood_df.head()

In [ ]:
altbank_data={"Neighborhood":[],
           "Alt Bank Name":[],
           "Alt Bank Lat":[],
           "Alt Bank Long": []
          }

In [ ]:
location_url = "https://maps.googleapis.com/maps/api/geocode/json?"

params = {
    "key": gkey,
}

In [ ]:
# use iterrows to iterate through pandas dataframe
for index, row in neighborhood_df.iterrows():

    # get restaurant type from df
    la_location = row['Neighborhood']

    # add keyword to params dict
    params['address'] = f"{la_location}+LA+CA"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {la_location}.")
    response = requests.get(location_url, params=params).json()
     
    try:
        print(f"Locating the lat,long coordinates for {la_location}")
        
        neighborhood_df.loc[index, "Neighborhood Lat"] = (response["results"][0]["geometry"]["location"]["lat"])
        neighborhood_df.loc[index, "Neighborhood Long"] = (response["results"][0]["geometry"]["location"]["lng"])
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
bank = 1
radius = 5000
params = {
    "radius": radius,
    "key": gkey
}

# use iterrows to iterate through pandas dataframe
for index, row in neighborhood_df.iterrows():
    neighbor = row["Neighborhood"]
    lat = row["Neighborhood Lat"]
    long = row["Neighborhood Long"]
    la_coords = f"{lat},{long}"

    count = 0
    page = 1
    current = 1

    
    print(f"{bank}. Locating the Alternative Banks in {neighbor}, CA")
    bank = bank + 1
    
    for i in range (0,3):
        try:
            print(f"Page: {page}")
            if count < 1:
        
                params['location'] = la_coords
                url_pass1 = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?keyword=check%20cashing"
    
                response1 = requests.get(url_pass1, params)
                bank_data1 = response1.json()
            
                for x in range(len(bank_data1["results"])):
                    altbank_data["Neighborhood"].append(neighbor)
                    altbank_data["Alt Bank Name"].append(bank_data1["results"][x]["name"])
                    altbank_data["Alt Bank Lat"].append(bank_data1["results"][x]["geometry"]["location"]["lat"])
                    altbank_data["Alt Bank Long"].append(bank_data1["results"][x]["geometry"]["location"]["lng"])
    
                time.sleep(2)
    
                count = count + 1
                current = current + 1
                page = page + 1
               
                try:
                    pagetoken = bank_data1["next_page_token"]
                except (KeyError, IndexError):
                    print(f"There are no more alternative banks in {neighbor}, CA.")
                    break
    
            else:
            
                params['location'] = la_coords
    
                url_pass2 = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?keyword=check%20cashing&pagetoken={pagetoken}"
    
                response2 = requests.get(url_pass2, params)
                bank_data2 = response2.json()
            
                for x in range(len(bank_data1["results"])):
                    altbank_data["Neighborhood"].append(neighbor)
                    altbank_data["Alt Bank Name"].append(bank_data2["results"][x]["name"])
                    altbank_data["Alt Bank Lat"].append(bank_data2["results"][x]["geometry"]["location"]["lat"])
                    altbank_data["Alt Bank Long"].append(bank_data2["results"][x]["geometry"]["location"]["lng"])
    
                time.sleep(2)
    
                count = count + 1
                page = page + 1
                current = current + 1
                
                try:
                    pagetoken = bank_data2["next_page_token"]
                except (KeyError, IndexError):
                    print(f"There are no more alternative banks in {neighbor}, CA.")
                    break
        except (KeyError, IndexError):
            print(f"There are no more alternative banks in {neighbor}, CA.")
            break
        
    print("------------")

In [ ]:
altbank_df=pd.DataFrame.from_dict(altbank_data, orient='index')
altbank_df = altbank_df.transpose()

altbank_df.head()

In [ ]:
altbank_df.to_csv(path_or_buf="LA_Alt_Banks.csv", sep=',')